In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Nov  10 20:03:47 2019

@author: JJB247
"""
"""
H added up path lengths along the boundaries
he used expaning sphere method
worked with computer science student who did fast march algorithm
One overall goal was to find the triple phase boundary density
"
The microstructural parameters such as volume fractions,
: DO NOT need particle diameter as its own quantity:, active/inactive triple 
phase boundary, tortuosity; were calculated
for the as-reduced cell and cells after operations.
"
"""
print('')

NOTES: 
this implementation will attempt to use k-dimensional trees to efficiently find nearest neighbor triple pairs

this may also involve using R-trees or R*-trees, in order to eliminate the need of preloading all the data

a cKDtree could be implemented for increased optomization in the future
    

learning:
https://observablehq.com/@2talltim/spatial-data-structures-octrees-bsp-and-k-d-trees


In [2]:
# Package Import
from threshold import threshold           # File I created to threshold images
from scipy.io import loadmat              # loads .mat file
from scipy import spatial                 # I'm not a math major but this works
from skimage import measure               # contains functions to find contours in 2d
import matplotlib.pyplot as plt           # plots data: includes plot & imshow
from mpl_toolkits import mplot3d          # supports 3d plotting
import pandas as pd                       # easy to use 2D data structure
import numpy as np                        # contains a variety of numerical
                                          # methods and data structures
from collections import Counter           # structure that counts occurences
from smt.sampling_methods import LHS      # latin hypercube sampling function
import pyDOE as pydoe                     # contains sampling methods
import os                                 # interacts with PATH to find files
import time                               # used for timer function
import tqdm                               # progress bar for iterables
import math                               # mathematical functions
import re                                 # regex bayBEEE

# define target folder
path = 'C:\\Users\\User\\Documents\\SOFC_Data_Science\\Data_OG'

In [3]:
# Load files from path 
# start timer
tic = time.time()
def load_files(path):
    """
    param: path: full path to the folder containing all data files
    return: loaded: dict of all files loaded, with keys equal to the
                    important part of the file names: format "PCT195-A2-Anode"
    return: filenames: list of full file paths for each file
    """
    filenames = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if '.mat' in file:
                filenames.append(os.path.join(r, file))

    print('files found: ')
    for f in filenames:
        print('.' + f[49:])

    # load all .mat files from target folder
    loaded={}
    for filename in filenames:
        fn = 'Amira_' + re.sub(r"[^\w\s\\:]", '_', filename)[50:]
        loaded.update({filename[50:-4] : loadmat(filename)[fn]})
    print(str(time.time()-tic) + ' seconds elapsed')
    return loaded, filenames
loaded,filenames = load_files(path)

files found: 
.\PCT195-A2-Anode.mat
.\PCt195-B1-Anode.mat
.\PCT195-B2-Anode-.mat
.\PCT209-1Khrs-anode.mat
.\PCT209-As-reduced-Anode.mat
1.661576509475708 seconds elapsed


In [4]:
# extract one file
onefile = loaded[filenames[0][50:-4]]

#allboundaries = []
black = []
white = []
grey  = []

for layerdepth in tqdm.tqdm(range(onefile.shape[2])):
#for layerdepth in [0]:
    # ========================================================================
    # ========================= FIND BORDERS =================================
    # ========================================================================
    currentlayer = onefile[:,:,layerdepth] # extract one sheet of data
    darkregions = measure.find_contours(currentlayer, 1)
    whiteregions = measure.find_contours(currentlayer, 2)
    
    # filter the image to highlight grey regions
    filteredlayer = currentlayer.copy()
    filteredlayer[filteredlayer == 3] = 1
    greyregions = measure.find_contours(filteredlayer, 1)
    # ========================================================================
    
    # add all black outlines (red lines) to a single list
    colordimension = 1
    for bsublist in darkregions:
        # iterate through every subarray in the black outlines
        for point in bsublist:
            # iterate through every point in the sublist
            # add the z coordinate and a fourth dimension (color)
            point = np.append(point,layerdepth)
            #point = np.append(point,colordimension)
            #allboundaries.append(point)
            black.append(point)
            
    colordimension = 3
    # add all white outlines (blue lines) to a single list
    for wsublist in whiteregions:
        for point in wsublist:
            point = np.append(point,layerdepth)
            #point = np.append(point,colordimension)
            #allboundaries.append(point)
            white.append(point)
    
    colordimension = 2
    # add all grey outlines (yellow lines) to a single list
    for ysublist in greyregions:
        for point in ysublist:
            point = np.append(point,layerdepth)
            #point = np.append(point,colordimension)
            #allboundaries.append(point)
            grey.append(point)

100%|████████████████████████████████████████████████████████████████████| 73/73 [00:26<00:00,  2.78it/s]


In [5]:
#bigtree = spatial.KDTree(allboundaries)
blacktree = spatial.KDTree(black)
whitetree = spatial.KDTree(white)
greytree  = spatial.KDTree(grey)

In [6]:
# "For each element self.data[i] of this tree, results[i] is a list of the indices of its neighbors in other.data."

# this works.

# for index i in close2, the points listed are the nearest neighbors
# BUT: every empty neighbor is a point where radius > 1: so, not a boundary
# now all I need to do is find the nearest neighbors between black and grey, 
# and pick out all the indexes where both lists do not have empty lists
tic = time.time()
bwneighbors = blacktree.query_ball_tree(whitetree, r=1, p=2.0, eps=0)
print('T1,T2: ' + str(time.time()-tic) +  ' seconds elapsed.')
tic = time.time()
bgneighbors = blacktree.query_ball_tree(greytree, r=1, p=2.0, eps=0)
print('T1,T3: ' + str(time.time()-tic) +  ' seconds elapsed.')

T1,T2: 467.571658372879 seconds elapsed.
T1,T3: 711.0746524333954 seconds elapsed.


In [7]:
i = 0
tpb = []
for i in tqdm.tqdm(range(len(bgneighbors))):
    if bwneighbors[i] and bgneighbors[i]:
        tpb.append(i)
        i+=1

100%|█████████████████████████████████████████████████████| 1235081/1235081 [00:00<00:00, 3183194.45it/s]
